## WSD using BERT Masked Language Model
This notebook explores the a part of the idea proposed by Ajit Rakasekharan in his blog post 
[Examining BERT raw embeddings.](https://towardsdatascience.com/examining-berts-raw-embeddings-fd905cb22df7) 

The idea is that examining the predictions of a masked language model for a masked ambiguous word can yield insights into the semantic meaning of the ambiguous word.

We use the HuggingFace BERT for Masked LM with weights from a bert-base-cased pre-trained model for our experiment.

We mask the ambiguous word (here we have used bank for our test) in sentences, and then send them through a BERT MLM model. Output is an array of logits for each position of the input sequence. So assuming a sentence with T tokens and a vocabulary size of V, the predictions of the MLM is (1, T, V) where 1 is the batch size (1 input sentence at a time in our experiment).

In order to find the top k predictions, the logits for the masked position is softmaxed and the top k values chosen.



## Prepare your environment

As always, we highly recommend that you install all packages with a virtual environment manager, like [venv](https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/) or [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html), to prevent version conflicts of different packages.  

### Masked LM Model and Tokenizer 
[tutorial](https://huggingface.co/docs/transformers/tasks/language_modeling)  
Task is to predict words that are masked using BERT, so we will use BERTMaskedLM model and BERTTokenizer and use the pre-trained bert-base-uncased model.

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForMaskedLM

In [2]:
device = torch.device("mps")

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased', return_dict=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We are going to use the pre-trained BERT language model in inference mode only.

The tokenizer tokenizes the input sequence and pads it with the [CLS] and [SEP] tokens.

The output produced by the model has two components, loss and logits. The logits component has shape (1, number_of_tokens, vocab_size) where the leading 1 represents the single input sentence.

We will identify the logits corresponding to the position of our masked token, identify the top 5 vocabulary words predicted for that position, and return the softmax probabilities for each of the top 5 predicted words.

In [4]:
inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
outputs = model(**inputs)

In [5]:
tokenizer.convert_ids_to_tokens(inputs.input_ids[0])


['[CLS]', 'The', 'capital', 'of', 'France', 'is', '[MASK]', '.', '[SEP]']

In [6]:
outputs

MaskedLMOutput(loss=None, logits=tensor([[[ -7.1545,  -6.9931,  -7.1826,  ...,  -5.9124,  -5.6733,  -5.9854],
         [ -8.0190,  -8.1319,  -8.0509,  ...,  -6.5679,  -6.4058,  -6.8998],
         [ -4.9772,  -6.1781,  -6.0669,  ...,  -5.6362,  -4.6603,  -5.1241],
         ...,
         [ -3.4420,  -3.2557,  -3.5733,  ...,  -2.4606,  -2.6495,  -3.1952],
         [-10.5890, -10.4621, -11.7181,  ...,  -7.4646,  -9.9543,  -8.3927],
         [-14.8900, -14.8873, -14.4569,  ..., -11.6588, -13.0151, -11.6073]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [7]:
def get_mask_index(input_ids, tokenizer):
    x = input_ids[0]
    is_masked = torch.where(x == tokenizer.mask_token_id, x, 0)
    mask_idx = torch.nonzero(is_masked)
    return mask_idx.item()

mask_idx = get_mask_index(inputs.input_ids, tokenizer)
mask_idx

6

In [8]:
def get_top_k_predictions(pred_logits, mask_idx, top_k):
    probs = torch.nn.functional.softmax(pred_logits[0, mask_idx, :], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)
    top_k_pct_weights = [100 * x.item() for x in top_k_weights]
    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices)
    return list(zip(top_k_tokens, top_k_pct_weights))


get_top_k_predictions(outputs.logits, mask_idx, 5)

[('Paris', 44.46823596954346),
 ('Lyon', 9.3959279358387),
 ('Toulouse', 8.234556019306183),
 ('Lille', 7.515136897563934),
 ('Marseille', 5.692216008901596)]

### WSD Test Sentences
We take our pair of sentences for disambiguating the word bank and mask them, and extract the top 20 predictions from the pre-trained BERT MLM model.

As expected, the first set of predictions predominantly point to some sort of financial institution, whereas the second set of predictions predominantly point to some geographical formation around bodies of water.

In [9]:
sentences = [
  "Go to the [MASK] and deposit your pay check.",
  "Jim and Janet went down to the river [MASK] to admire the swans."
]

In [10]:
def get_predictions(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    mask_idx = get_mask_index(inputs.input_ids, tokenizer)
    top_preds = get_top_k_predictions(outputs.logits, mask_idx, 20)
    return top_preds

In [11]:
get_predictions(sentences[0], tokenizer, model)

[('bank', 70.31407952308655),
 ('office', 10.280513763427734),
 ('register', 1.7451928928494453),
 ('store', 1.6284745186567307),
 ('bathroom', 0.9394743479788303),
 ('library', 0.8934773504734039),
 ('desk', 0.8724354207515717),
 ('counter', 0.7977306842803955),
 ('hotel', 0.5163747351616621),
 ('lobby', 0.4956936929374933),
 ('kitchen', 0.36370845045894384),
 ('garage', 0.34799131099134684),
 ('door', 0.34127223771065474),
 ('car', 0.3311340929940343),
 ('house', 0.26490448508411646),
 ('airport', 0.2547027077525854),
 ('elevator', 0.24911381769925356),
 ('back', 0.2480770694091916),
 ('computer', 0.24019242264330387),
 ('banks', 0.2349138492718339)]

In [12]:
get_predictions(sentences[1], tokenizer, model)

[('##bank', 32.60221481323242),
 ('below', 13.03187757730484),
 ('bank', 11.94089949131012),
 (',', 5.626518651843071),
 ('##boat', 3.163899853825569),
 ('##front', 2.733191102743149),
 ('basin', 1.6210585832595825),
 ('##bed', 1.2178393080830574),
 ('together', 1.1841688305139542),
 ('bed', 0.9657183662056923),
 ('again', 0.8369838818907738),
 ('deck', 0.8356185629963875),
 ('valley', 0.7271412294358015),
 ('mouth', 0.7227532099932432),
 ('boat', 0.7151085417717695),
 ('pier', 0.649333419278264),
 ('house', 0.6301531102508307),
 ('banks', 0.5700564943253994),
 ('pool', 0.5345729645341635),
 ('Thames', 0.4995543509721756)]

## Assignment
In this week's assignment, you are tasked with processing SemCor data and feed the data into BERT masked-LM. After that, use the predictions to find the most likely sense of the target word using WordNet similarity.

### Data Preprocessing 
You can find a sample of SemCor dataset [here](https://drive.google.com/file/d/1inmv3rUcGrtiS4VQwTMsT9HF-iL8jc5V/view?usp=sharing) and load the data using the following methods.

In [53]:
import json
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
sents = []
tokens = []
wn_id = []
lemmatizer = WordNetLemmatizer()

with open('semcor.sample.jsonl') as f:
    for line in f:
        data = json.loads(line)
        sents.append(data['sent'])
        tokens.append(data['tokens'])
        wn_id.append(data['wnid'])


In [14]:
print(sents[10])
print(tokens[10])
print(wn_id[10])

implementation of georgia 's automobile title law was also recommended by the outgoing jury . 
['implementation', 'of', 'georgia', "'s", 'automobile', 'title', 'law', 'was', 'also', 'recommended', 'by', 'the', 'outgoing', 'jury', '.']
['implementation%1:04:01::', 0, 'georgia%1:15:00::', 0, 'automobile%1:06:00::', 'title%1:10:04::', 'law%1:10:00::', 0, 'also%4:02:00::', 'recommend%2:32:01::', 0, 0, 'outgoing%3:00:00::', 'jury%1:14:00::', 0]


In [15]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/rakson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/rakson/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [64]:
# The WordNet ID can be converted to NLTK Lemma using the following function
wn.lemma_from_key('implementation%1:04:01::')

Lemma('execution.n.06.implementation')

### TODO 
Please implement a method to convert the data to BERT Masked-LM format and keep track of the headword. Store the data into the following lists

word[i] = 'implementation'  
ground_truth[i] = 'implementation%1:04:01::'  
sent[i] = "[MASK] of georgia 's automobile title law was also recommended by the outgoing jury ."  



In [82]:
word = []
ground_truth = []
sent = []
for i in range(len(sents)):
    for j in range(len(wn_id[i])):
        if (wn_id[i][j] != 0):
            word.append(tokens[i][j])
            ground_truth.append(wn_id[i][j])
            sent.append(sents[i].replace(tokens[i][j], '[MASK]'))

In [50]:
print(word[0])
print(ground_truth[0])
print(sent[0])

said
say%2:32:00::
the fulton_county_grand_jury [MASK] friday an investigation of atlanta 's recent primary_election produced " no evidence " that any irregularities took_place . 


#### Identify the top 5 predictions other than the headword using Masked-LM 
1. Use get_predictions to get the predicted words
2. Use lemmatizer to lemmatize the prediction
3. Remove headword
4. Keep top 5 unique predictions

In [76]:
import pandas as pd
candidate_lemmas = []
for i in range(len(word)):
    df = pd.DataFrame(get_predictions(sent[0], tokenizer, model)[1:6], columns=['word', 'probability'])
    for word in df['word']:
        df['word', i] = lemmatizer.lemmatize(word)
    candidate_lemmas.append(df['word'].tolist())

In [77]:
candidate_lemmas[0]

['found', 'reported', ',', 'told', '_']

example:  
candidate_lemmas = ['office', 'register', 'store', 'bathroom', 'library']


Identify the most similar sense of headword with relation to the 5 unique candidates

In [ ]:
predicted_sense = [[wn.synsets(wn.lemma_from_key(word))] for word in ground_truth]

In [101]:
#lemma to string
#wup similarity
wn.lemma_from_key('say%2:32:00::').key()

'say%2:32:00::'

In [91]:
ground_truth[0]

'say%2:32:00::'

In [89]:
predicted_sense[0]

[[]]

For evaluation purpose, for i = 50, please run the process and print out the following:  
1. word[50]
2. ground_truth[50] (in synset or lemma)
3. sent[50]
4. candidate_lemmas
5. predicted_sense (in synset or lemma)    

Also, please print out the accuracy of the process over our dataset

In [ ]:
score = ...

## TA's Note

Congratulations, you made it to the end of the tutorial! Make sure you make an appointment to show your work and turn in your finished assignment before next week's lesson. We will ask you to run your code, so double check that everything is working and that your model is saved. Don't worry if you didn't pass the evaluation requirements, you'll still get partial points for trying.